In [1]:
# Betweenness Centrality
import data_analysis_tools as da
users = [
{ "id": 0, "name": "Hero" },
{ "id": 1, "name": "Dunn" },
{ "id": 2, "name": "Sue" },
{ "id": 3, "name": "Chi" },
{ "id": 4, "name": "Thor" },
{ "id": 5, "name": "Clive" },
{ "id": 6, "name": "Hicks" },
{ "id": 7, "name": "Devin" },
{ "id": 8, "name": "Kate" },
{ "id": 9, "name": "Klein" }
]

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
(4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [2]:
for user in users:
    user["friends"] = []

for i, j in friendships:
    users[i]["friends"].append(users[j])
    users[j]["friends"].append(users[i])

In [3]:
from collections import deque

def shortest_paths_from(from_user):

    # a dictionary from "user_id" to all shortest paths to that user
    shortest_paths_to = { from_user["id"]: [[]] }

    # a queue of (previous user, next user) that we need to check
    # starts out with all pairs (from_user, friend_of_from_user)
    frontier = deque((from_user, friend) for friend in from_user["friends"])

    # keep going until we empty the queue

    while frontier:
        prev_user, user = frontier.popleft() # remove the user who's first in the queue
        user_id = user["id"]

        # because of the way we're adding to the queue
        # necessarily we already know some shortest paths to prev_user
        paths_to_prev_user = shortest_paths_to[prev_user["id"]]
        new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]

        # its possible that we already know the shortest path
        old_paths_to_user = shortest_paths_to.get(user_id, [])

        # what's the shortest path to here that we've seen so far?
        if old_paths_to_user:
            min_path_len = len(old_paths_to_user[0])
        else:
            min_path_len = float('inf')

        # only keep paths that aren't too long and are actually new
        new_paths_to_user = [
            path for path in new_paths_to_user
            if len(path) <= min_path_len
            and path not in old_paths_to_user
        ]

        shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user

        # add never-seen neighbors to the frontier
        frontier.extend((user, friend) for friend in user["friends"]
            if friend["id"] not in shortest_paths_to
        )
    
    return shortest_paths_to

In [4]:
for user in users:
    user["shortest_paths"] = shortest_paths_from(user)

In [5]:
users[0]["shortest_paths"]

{0: [[]],
 1: [[1]],
 2: [[2]],
 3: [[1, 3], [2, 3]],
 4: [[1, 3, 4], [2, 3, 4]],
 5: [[1, 3, 4, 5], [2, 3, 4, 5]],
 6: [[1, 3, 4, 5, 6], [2, 3, 4, 5, 6]],
 7: [[1, 3, 4, 5, 7], [2, 3, 4, 5, 7]],
 8: [[1, 3, 4, 5, 6, 8],
  [2, 3, 4, 5, 6, 8],
  [1, 3, 4, 5, 7, 8],
  [2, 3, 4, 5, 7, 8]],
 9: [[1, 3, 4, 5, 6, 8, 9],
  [2, 3, 4, 5, 6, 8, 9],
  [1, 3, 4, 5, 7, 8, 9],
  [2, 3, 4, 5, 7, 8, 9]]}

In [6]:
for user in users:
    user["betweenness_centrality"] = 0.0

for source in users:
    source_id = source["id"]
    for target_id, paths in source["shortest_paths"].items():
        if source_id < target_id: # this is for not counting twice
            num_paths = len(paths)
            contrib = 1 / num_paths
            for path in paths:
                for id in path:
                    if id not in [source_id, target_id]:
                        users[id]["betweenness_centrality"] += contrib

In [7]:
for user in users:
    print(user["id"], user["betweenness_centrality"]) 

0 0.0
1 3.5
2 3.5
3 18.0
4 20.0
5 20.5
6 6.0
7 6.0
8 8.5
9 0.0


In [8]:
# closeness centrality

def farness(user):
    return sum(len(paths[0]) for paths in user["shortest_paths"].values())

for user in users:
    user["closeness_centrality"] = 1 / farness(user)

In [9]:
for user in users:
    print(user["id"], user["closeness_centrality"])

0 0.029411764705882353
1 0.037037037037037035
2 0.037037037037037035
3 0.045454545454545456
4 0.05
5 0.05
6 0.041666666666666664
7 0.041666666666666664
8 0.03571428571428571
9 0.027777777777777776


In [10]:
"""
    As we saw, computing shortest paths is kind of a pain. For this reason, betweenness and
    closeness centrality aren’t often used on large networks. The less intuitive (but generally
    easier to compute) eigenvector centrality is more frequently used.
"""
from functools import partial
# eigenvector centrality

# matrix multiplication
def matrix_product_entry(A, B, i, j):
    return da.dot_product(A[i], da.get_column(B, j))


def matrix_multiply(A, B):
    n1, k1 = len(A), len(A[0])
    n2, k2 = len(B), len(B[0])

    if k1 != n2:
        raise ArithmeticError('incompatible shapes!')
    else:
        return da.make_matrix(n1, k2, partial(matrix_product_entry, A, B))

In [11]:
mat1 = [[1, 2],
[2, 3],
[3, 1]]

mat2 = [[3, 3, 0],
[0, 1, 2]]

matrix_multiply(mat1, mat2)

[[3, 5, 4], [6, 9, 6], [9, 10, 2]]

In [12]:
def vector_to_matrix(v):
    """converts list to nx1 matrix"""
    return [[vi] for vi in v]

def vector_from_matrix(mat):
    """converts nx1 matrix to list"""
    return [row[0] for row in mat]


mat1 = [[1, 3, 3],
    [1, 5, -1],
    [3, 1, 0]]
mat = vector_to_matrix([1, 2, 3])
vec = vector_from_matrix(mat)


print(mat)
print(vec)

def matrix_operate(A, v):
    v_as_matrix = vector_to_matrix(v)
    product = matrix_multiply(A, v_as_matrix)
    return vector_from_matrix(product)


[[1], [2], [3]]
[1, 2, 3]


In [13]:
def find_eigenvector(A, tolerance=0.00001):

    guess = [da.random.random() for _ in A]
    while True:
        result = matrix_operate(A, guess)
        length = da.magnitude(result)
        next_guess = da.scalar_multiply(1/length, result)

        if da.euclidean(guess, next_guess) < tolerance:
            return next_guess, length # eigenvector, eigenvalue
            
        guess = next_guess

In [17]:
"""
    note:
    rotate = [[ 0, 1],
            [-1, 0]]    doesn't have an eigenvector of real values

    flip = [[0, 1],     eigenvector is [1, 1] but if you start with random numbers it will flip [x, y] -> [y, x] -> [x, y] and run forever.
        [1, 0]]
"""

mat = [[3, 2],
    [3, 5]]
find_eigenvector(mat, 0.001)

([0.4808746347547578, 0.8767893621899608], 6.645243791650158)

In [26]:
def entry_fn(i, j):
    return 1 if (i, j) in friendships or (j, i) in friendships else 0

n = len(users)

adjacency_matrix = da.make_matrix(n, n, entry_fn)
adjacency_matrix

[[0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]

In [27]:
eigenvector_centralities, _ = find_eigenvector(adjacency_matrix)

In [29]:
eigenvector_centralities

[0.38578114702039273,
 0.5147900288654446,
 0.5147900288654446,
 0.47331408747048337,
 0.23360714815327532,
 0.15014820695148395,
 0.08355082059776549,
 0.08355082059776549,
 0.07284228529142327,
 0.027292344648162888]

In [30]:
# directed graphs and PageRank

endorsements = [(0, 1), (1, 0), (0, 2), (2, 0), (1, 2),
(2, 1), (1, 3), (2, 3), (3, 4), (5, 4),
(5, 6), (7, 5), (6, 8), (8, 7), (8, 9)]

for user in users:
    user["endorses"] = []
    user["endorsed_by"] = []

for source_id, target_id in endorsements:
    users[source_id]["endorses"].append(users[target_id])
    users[target_id]["endorsed_by"].append(users[source_id])

In [40]:
endorsements_by_id = [(user["id"], len(user["endorsed_by"])) for user in users]

sorted(endorsements_by_id, key=lambda e: e[1], reverse=True)

[(0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [43]:
def page_rank(users, damping=0.85, num_iters=100):
    # initially distribute PageRank evenly
    num_users = len(users)
    pr = { user["id"]: 1/num_users for user in users }

    # this is the small fraction of PageRank
    # that each node gets each iteration
    base_br = (1 - damping) / num_users

    for _ in range(num_iters):
        next_pr = { user["id"]: base_br for user in users }
        
        for user in users:
            # distribute PageRank to outgoing links
            links_pr = pr[user["id"]] * damping
            for endorsee in user["endorses"]:
                next_pr[endorsee["id"]] += links_pr / len(user["endorses"])
        
        pr = next_pr
    return pr

In [61]:
prs = page_rank(users, 0.8)

sorted(prs.items(), key=lambda x: x[1], reverse=True)

[(4, 0.08015082956259427),
 (1, 0.05384615384615385),
 (2, 0.05384615384615385),
 (5, 0.052941176470588235),
 (8, 0.052941176470588235),
 (0, 0.04871794871794872),
 (3, 0.04871794871794872),
 (6, 0.041176470588235294),
 (7, 0.041176470588235294),
 (9, 0.041176470588235294)]

In [62]:
prs

{0: 0.04871794871794872,
 1: 0.05384615384615385,
 2: 0.05384615384615385,
 3: 0.04871794871794872,
 4: 0.08015082956259427,
 5: 0.052941176470588235,
 6: 0.041176470588235294,
 7: 0.041176470588235294,
 8: 0.052941176470588235,
 9: 0.041176470588235294}